## 1- Load GeoJSON and Shapefile Data:

In [ ]:
import geopandas as gpd
import folium

# Load the GeoJSON file
geojson_path = '/path_to_file/earthquake.geojson'
gdf = gpd.read_file(geojson_path)

# Load the shapefile
shapefile_path = '/path_to_file/NZ_Shapefiles/territorial-authority-2023-generalised.shp'
regions_gdf = gpd.read_file(shapefile_path)


## 2- Reproject Data:
Ensure both datasets use the same CRS (Coordinate Reference System):

In [ ]:
# Check and reproject GeoJSON CRS to match shapefile
if gdf.crs != "EPSG:4326":
    gdf = gdf.to_crs("EPSG:4326")
# Reproject shapefile to EPSG:4326
if regions_gdf.crs != "EPSG:4326":
    regions_gdf = regions_gdf.to_crs("EPSG:4326")


## 3- Spatial Join and Aggregation:
Assign earthquakes to regions and count occurrences:

In [ ]:
# Perform spatial join: Assign earthquakes to regions
joined_gdf = gpd.sjoin(gdf, regions_gdf, how="inner", predicate="within")

# Aggregate earthquake counts by region
earthquake_counts = joined_gdf.groupby('TA2023_V1_').size().reset_index(name='earthquake_count')


## 4- Merge and Clean Data:
Add earthquake counts to the shapefile data:

In [ ]:
# Merge the counts back with the shapefile GeoDataFrame
regions_gdf = regions_gdf.merge(earthquake_counts, on="TA2023_V1_", how="left")


# Remove rows where TA2023_V1_ is 999
regions_gdf = regions_gdf[regions_gdf["TA2023_V1_"] != "999"]

# Fill NaN values with 0 for regions without earthquakes
regions_gdf["earthquake_count"] = regions_gdf["earthquake_count"].fillna(0)

# Save cleaned GeoJSON for visualization
output_geojson_path = '/path_to_save_file/joined_data_cleaned.geojson'
regions_gdf.to_file(output_geojson_path, driver="GeoJSON")

# Load the cleaned GeoJSON for mapping
gdf1 = gpd.read_file(output_geojson_path)

## 5- Create Choropleth Map:
Generate an interactive map with Folium:

In [ ]:
# Initialize the map centered around New Zealand
m = folium.Map(location=[-40.9006, 174.8860], zoom_start=5)

# Add the Choropleth layer
folium.Choropleth(
    geo_data=output_geojson_path,
    name='choropleth',
    data=gdf1,
    columns=['TA2023_V1_', 'earthquake_count'],  # Unique region identifier
    key_on='feature.properties.TA2023_V1_',
    fill_color='Reds',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Earthquake Count'
).add_to(m)

# Add a layer control
folium.LayerControl().add_to(m)

# Save the map to an HTML file
map_output_path = "earthquake_choropleth_map.html"
m.save(map_output_path)
print(f"Choropleth map saved as {map_output_path}")
 